In [ ]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting

### since I already have the training dataset split, I just need to train!

In [ ]:
dataset = "data"
NUM_CLASSES = 3
# model_save_path = "model/"

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [ ]:
with tf.device():
    model = Sequential([
        Dense(units=32, activation='relu', input_shape=(42,)),
        Dense(units=64, activation='relu',
            kernel_regularizer=keras.regularizers.l1_l2(0.05)),
        Dense(units=256, activation='relu'),
        Dropout(0.3),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=16, activation='relu', 
            kernel_regularizer=keras.regularizers.l1_l2(0.01)),
        Dropout(0.3),
        Dense(units=NUM_CLASSES, activation='softmax')
])

In [ ]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [ ]:
#fitting
with tf.device():
    history = model.fit(X_train,Y_train, epochs=50, batch_size=32, validation_data=(X_val,Y_val), callbacks=[checkpoint])